# 02 - Feature Engineering | SweetReturns Golden City

**Notebook 2 of 5** in the SweetReturns data pipeline.

This notebook reads the cleaned parquet from notebook 01 and computes all technical/quantitative features:
- Drawdown features
- Volatility features
- Volume features
- Momentum indicators (RSI, MACD, Bollinger Bands, OBV, ATR, Stochastic, ROC, MFI)
- Hurst exponent
- SPY benchmark + relative features
- Graph/network features (correlation matrix + centrality)
- HMM regime detection

In [ ]:
!pip install -q pandas numpy scipy hmmlearn scikit-learn networkx python-louvain

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

df = pd.read_parquet("stock_data_clean.parquet")
print(f"Loaded: {df.shape}, Tickers: {df['ticker'].nunique()}")
df = df.sort_values(["ticker", "Date"]).reset_index(drop=True)

In [ ]:
# Compute drawdown from all-time high (per ticker)
def compute_drawdown_features(group):
    close = group["Close"]
    expanding_max = close.expanding().max()
    group["drawdown_pct"] = (close - expanding_max) / expanding_max
    group["drawdown_percentile"] = group["drawdown_pct"].rank(pct=True)
    return group

df = df.groupby("ticker", group_keys=False).apply(compute_drawdown_features)
print(f"Drawdown stats:\n{df['drawdown_pct'].describe()}")

In [ ]:
def compute_volatility_features(group):
    ret = group["daily_return"]
    group["realized_vol_20d"] = ret.rolling(20).std() * np.sqrt(252)
    group["realized_vol_60d"] = ret.rolling(60).std() * np.sqrt(252)
    
    # Vol percentile: where current vol sits vs its own 1-year history
    vol = group["realized_vol_20d"]
    group["vol_percentile"] = vol.rolling(252, min_periods=60).apply(
        lambda x: stats.percentileofscore(x[:-1], x.iloc[-1]) / 100 if len(x) > 1 else 0.5,
        raw=False
    )
    
    # Vol term structure
    group["vol_term_structure"] = group["realized_vol_20d"] / group["realized_vol_60d"].replace(0, np.nan)
    
    return group

df = df.groupby("ticker", group_keys=False).apply(compute_volatility_features)
print(f"Vol stats:\n{df[['realized_vol_20d', 'vol_percentile']].describe()}")

In [ ]:
def compute_volume_features(group):
    vol = group["Volume"].astype(float)
    group["volume_percentile"] = vol.rolling(252, min_periods=60).apply(
        lambda x: stats.percentileofscore(x[:-1], x.iloc[-1]) / 100 if len(x) > 1 else 0.5,
        raw=False
    )
    return group

df = df.groupby("ticker", group_keys=False).apply(compute_volume_features)
print(f"Volume percentile stats:\n{df['volume_percentile'].describe()}")

In [ ]:
def compute_momentum_features(group):
    close = group["Close"]
    high = group["High"]
    low = group["Low"]
    volume = group["Volume"].astype(float)
    ret = group["daily_return"]
    
    # RSI 14
    delta = close.diff()
    gain = delta.clip(lower=0)
    loss = (-delta).clip(lower=0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / avg_loss.replace(0, np.nan)
    group["rsi_14"] = 100 - (100 / (1 + rs))
    
    # MACD (12, 26, 9)
    ema12 = close.ewm(span=12, adjust=False).mean()
    ema26 = close.ewm(span=26, adjust=False).mean()
    group["macd_line"] = ema12 - ema26
    group["macd_signal"] = group["macd_line"].ewm(span=9, adjust=False).mean()
    group["macd_histogram"] = group["macd_line"] - group["macd_signal"]
    
    # Bollinger Bands (20, 2)
    bb_mid = close.rolling(20).mean()
    bb_std = close.rolling(20).std()
    group["bb_upper"] = bb_mid + 2 * bb_std
    group["bb_lower"] = bb_mid - 2 * bb_std
    group["bb_width"] = (group["bb_upper"] - group["bb_lower"]) / bb_mid
    group["bb_pctb"] = (close - group["bb_lower"]) / (group["bb_upper"] - group["bb_lower"]).replace(0, np.nan)
    
    # OBV
    obv = (np.sign(close.diff()) * volume).fillna(0).cumsum()
    group["obv"] = obv
    # OBV slope (20-day linear regression slope)
    group["obv_slope_20d"] = obv.rolling(20).apply(
        lambda x: np.polyfit(range(len(x)), x, 1)[0] if len(x) == 20 else 0, raw=False
    )
    
    # ATR 14
    tr = pd.concat([
        high - low,
        (high - close.shift(1)).abs(),
        (low - close.shift(1)).abs()
    ], axis=1).max(axis=1)
    group["atr_14"] = tr.rolling(14).mean()
    group["atr_percentile"] = group["atr_14"].rolling(252, min_periods=60).apply(
        lambda x: stats.percentileofscore(x[:-1], x.iloc[-1]) / 100 if len(x) > 1 else 0.5,
        raw=False
    )
    
    # Stochastic Oscillator (14, 3)
    low14 = low.rolling(14).min()
    high14 = high.rolling(14).max()
    group["stoch_k"] = 100 * (close - low14) / (high14 - low14).replace(0, np.nan)
    group["stoch_d"] = group["stoch_k"].rolling(3).mean()
    
    # Rate of Change
    group["roc_5"] = close.pct_change(5)
    group["roc_20"] = close.pct_change(20)
    group["roc_60"] = close.pct_change(60)
    
    # MFI 14
    typical_price = (high + low + close) / 3
    mf = typical_price * volume
    pos_mf = mf.where(typical_price > typical_price.shift(1), 0).rolling(14).sum()
    neg_mf = mf.where(typical_price <= typical_price.shift(1), 0).rolling(14).sum()
    group["mfi_14"] = 100 - (100 / (1 + pos_mf / neg_mf.replace(0, np.nan)))
    
    # Z-scores
    group["zscore_20d"] = (ret - ret.rolling(20).mean()) / ret.rolling(20).std().replace(0, np.nan)
    group["zscore_60d"] = (ret - ret.rolling(60).mean()) / ret.rolling(60).std().replace(0, np.nan)
    
    # Autocorrelation (60-day rolling, lag 1)
    group["autocorrelation_lag1"] = ret.rolling(60).apply(
        lambda x: x.autocorr(lag=1) if len(x) == 60 else 0, raw=False
    )
    
    return group

df = df.groupby("ticker", group_keys=False).apply(compute_momentum_features)
print(f"Feature columns: {len(df.columns)}")
print(df[["rsi_14", "macd_histogram", "bb_pctb", "stoch_k", "mfi_14"]].describe())

In [ ]:
def compute_hurst(series, max_lag=20):
    """Compute Hurst exponent via R/S analysis."""
    lags = range(2, max_lag)
    tau = []
    for lag in lags:
        chunks = [series[i:i+lag] for i in range(0, len(series) - lag, lag)]
        rs_values = []
        for chunk in chunks:
            if len(chunk) < 2:
                continue
            mean_val = np.mean(chunk)
            cumdev = np.cumsum(chunk - mean_val)
            R = np.max(cumdev) - np.min(cumdev)
            S = np.std(chunk, ddof=1)
            if S > 0:
                rs_values.append(R / S)
        if rs_values:
            tau.append(np.mean(rs_values))
        else:
            tau.append(np.nan)
    
    valid = [(l, t) for l, t in zip(lags, tau) if not np.isnan(t) and t > 0]
    if len(valid) < 3:
        return 0.5
    log_lags = np.log([v[0] for v in valid])
    log_tau = np.log([v[1] for v in valid])
    slope, _, _, _, _ = stats.linregress(log_lags, log_tau)
    return np.clip(slope, 0, 1)

def compute_hurst_rolling(group):
    ret = group["daily_return"].values
    hurst = np.full(len(ret), np.nan)
    for i in range(252, len(ret)):
        hurst[i] = compute_hurst(ret[i-252:i])
    group["hurst_exponent"] = hurst
    return group

df = df.groupby("ticker", group_keys=False).apply(compute_hurst_rolling)
print(f"Hurst exponent stats:\n{df['hurst_exponent'].describe()}")

In [ ]:
# Download SPY data using yfinance as fallback, or use from dataset if present
spy_data = df[df["ticker"] == "SPY"][["Date", "Close", "daily_return"]].copy()
if len(spy_data) == 0:
    print("SPY not in dataset, using mean market return as proxy")
    market_return = df.groupby("Date")["daily_return"].mean().reset_index()
    market_return.columns = ["Date", "spy_return"]
else:
    spy_data = spy_data.rename(columns={"daily_return": "spy_return", "Close": "spy_close"})
    market_return = spy_data[["Date", "spy_return"]]

df = df.merge(market_return, on="Date", how="left")

# 20-day relative return vs SPY
df["stock_return_20d"] = df.groupby("ticker")["Close"].pct_change(20)
df["spy_return_20d"] = df["spy_return"].rolling(20).sum()  # approximate
df["relative_return_vs_spy"] = df["stock_return_20d"] - df["spy_return_20d"]

# Beta (252-day rolling)
# Simplified: we'll compute sector-relative z-score instead
def compute_sector_zscore(group):
    ret20 = group["stock_return_20d"]
    group["sector_zscore"] = (ret20 - ret20.mean()) / ret20.std().replace(0, np.nan)
    return group

df = df.groupby(["sector", "Date"], group_keys=False).apply(compute_sector_zscore)
print(f"Relative features computed. Columns: {len(df.columns)}")

In [ ]:
import networkx as nx
try:
    from community import community_louvain
    HAS_LOUVAIN = True
except ImportError:
    HAS_LOUVAIN = False
    print("python-louvain not available, skipping community detection")

# Use most recent 252 days for correlation
latest_date = df["Date"].max()
recent = df[df["Date"] > latest_date - pd.Timedelta(days=365)]
pivot = recent.pivot_table(index="Date", columns="ticker", values="daily_return")
corr_matrix = pivot.corr()

# Build graph from high correlations
G = nx.Graph()
tickers = corr_matrix.columns.tolist()
G.add_nodes_from(tickers)
for i, t1 in enumerate(tickers):
    for j, t2 in enumerate(tickers):
        if j > i:
            w = corr_matrix.iloc[i, j]
            if abs(w) > 0.5:
                G.add_edge(t1, t2, weight=w)

degree_cent = nx.degree_centrality(G)
betweenness_cent = nx.betweenness_centrality(G)
try:
    eigenvector_cent = nx.eigenvector_centrality(G, max_iter=1000)
except:
    eigenvector_cent = {t: 0 for t in tickers}
pagerank = nx.pagerank(G)

# Community detection
if HAS_LOUVAIN:
    communities = community_louvain.best_partition(G)
else:
    communities = {t: 0 for t in tickers}

# Map back to DataFrame
graph_features = pd.DataFrame({
    "ticker": tickers,
    "degree_centrality": [degree_cent.get(t, 0) for t in tickers],
    "betweenness_centrality": [betweenness_cent.get(t, 0) for t in tickers],
    "eigenvector_centrality": [eigenvector_cent.get(t, 0) for t in tickers],
    "pagerank": [pagerank.get(t, 0) for t in tickers],
    "community_id": [communities.get(t, 0) for t in tickers],
})
graph_features["community_size"] = graph_features["community_id"].map(
    graph_features.groupby("community_id")["ticker"].transform("count")
)

df = df.merge(graph_features, on="ticker", how="left")
print(f"Graph features added. Shape: {df.shape}")
print(f"Communities found: {graph_features['community_id'].nunique()}")
print(f"Edges in graph: {G.number_of_edges()}")

In [ ]:
from hmmlearn.hmm import GaussianHMM

# Prepare SPY features for regime detection
spy_df = df[df["ticker"] == df["ticker"].iloc[0]][["Date", "spy_return"]].drop_duplicates("Date").sort_values("Date").dropna()
if len(spy_df) < 100:
    spy_df = df.groupby("Date")["daily_return"].mean().reset_index()
    spy_df.columns = ["Date", "spy_return"]
    spy_df = spy_df.sort_values("Date").dropna()

spy_vol = spy_df["spy_return"].rolling(20).std() * np.sqrt(252)
hmm_features = pd.DataFrame({
    "return": spy_df["spy_return"].values,
    "vol": spy_vol.values,
}).dropna()

# Fit 4-state HMM
model = GaussianHMM(n_components=4, covariance_type="full", n_iter=200, random_state=42)
model.fit(hmm_features.values)
regimes = model.predict(hmm_features.values)

# Label regimes by mean return and vol
regime_stats = pd.DataFrame(hmm_features.values, columns=["return", "vol"])
regime_stats["regime"] = regimes
regime_summary = regime_stats.groupby("regime").agg({"return": "mean", "vol": "mean"})
print("Regime summary:\n", regime_summary)

# Map to semantic labels
regime_labels = {}
for r in regime_summary.index:
    bull = regime_summary.loc[r, "return"] > 0
    quiet = regime_summary.loc[r, "vol"] < regime_summary["vol"].median()
    if bull and quiet: regime_labels[r] = "bull_quiet"
    elif bull and not quiet: regime_labels[r] = "bull_volatile"
    elif not bull and quiet: regime_labels[r] = "bear_quiet"
    else: regime_labels[r] = "bear_volatile"

# Merge regime back to dates
aligned_dates = spy_df["Date"].iloc[19:].reset_index(drop=True)  # offset by rolling window
regime_df = pd.DataFrame({"Date": aligned_dates.values[:len(regimes)], "regime_label": [regime_labels[r] for r in regimes]})
df = df.merge(regime_df, on="Date", how="left")
df["regime_label"] = df["regime_label"].fillna("unknown")
print(f"\nRegime distribution:\n{df['regime_label'].value_counts()}")

In [ ]:
# Save feature-rich dataset
df.to_parquet("stock_features.parquet", index=False)
corr_matrix.to_parquet("correlation_matrix.parquet")
graph_features.to_parquet("graph_features.parquet")
print(f"Saved stock_features.parquet: {df.shape} ({len(df.columns)} columns)")
print(f"Saved correlation_matrix.parquet: {corr_matrix.shape}")
print(f"\nAll columns:\n{list(df.columns)}")
print(f"\nReady for 03_golden_tickets.ipynb!")

## Summary

This notebook computed the following feature categories:

| Category | Features |
|----------|----------|
| **Drawdown** | `drawdown_pct`, `drawdown_percentile` |
| **Volatility** | `realized_vol_20d`, `realized_vol_60d`, `vol_percentile`, `vol_term_structure` |
| **Volume** | `volume_percentile` |
| **RSI** | `rsi_14` |
| **MACD** | `macd_line`, `macd_signal`, `macd_histogram` |
| **Bollinger Bands** | `bb_upper`, `bb_lower`, `bb_width`, `bb_pctb` |
| **OBV** | `obv`, `obv_slope_20d` |
| **ATR** | `atr_14`, `atr_percentile` |
| **Stochastic** | `stoch_k`, `stoch_d` |
| **Rate of Change** | `roc_5`, `roc_20`, `roc_60` |
| **MFI** | `mfi_14` |
| **Z-scores** | `zscore_20d`, `zscore_60d` |
| **Autocorrelation** | `autocorrelation_lag1` |
| **Hurst** | `hurst_exponent` |
| **Relative** | `spy_return`, `stock_return_20d`, `spy_return_20d`, `relative_return_vs_spy`, `sector_zscore` |
| **Graph/Network** | `degree_centrality`, `betweenness_centrality`, `eigenvector_centrality`, `pagerank`, `community_id`, `community_size` |
| **Regime** | `regime_label` |

### Output files
- `stock_features.parquet` - Full dataset with all features
- `correlation_matrix.parquet` - Ticker-to-ticker correlation matrix
- `graph_features.parquet` - Network centrality features per ticker

### Next step
Proceed to **03_golden_tickets.ipynb** for signal generation and stock selection.